# Cookbook 2: Validate data during ingestion (take action on failures)

This cookbook showcases a sample GX data validation workflow characteristic of data ingestion at the start of the data pipeline. Data is loaded into a Pandas dataframe, cleaned, validated, and then ingested into a Postgres database table. This cookbook explores the validation workflow first in a notebook setting, then embedded within an Airflow pipeline.

This cookbook features a scenario in which a subset of data fails validation and must be handled in the pipeline.

This cookbook builds on [Cookbook 1: Validate data during ingestion (happy path)](Cookbook_1_Validate_data_during_ingestion_happy_path.ipynb) and focuses on how data validation failures can be programmatically handled in the pipeline based on GX Validation Results. This cookbook assumes basic familiarity with GX Core workflows; for a step-by-step explanation of the GX data validation workflow, refer to [Cookbook 1](Cookbook_1_Validate_data_during_ingestion_happy_path.ipynb). 

## Imports

This tutorial features the `great_expectations` library.

The `tutorial_code` module contains helper functions used within this notebook and the associated Airflow pipeline.

The `airflow_dags` submodule is included so that you can inspect the code used in the related Airflow DAG directly from this notebook.

In [1]:
import pathlib
import inspect

import great_expectations as gx
import great_expectations.expectations as gxe
import pandas as pd

import tutorial_code as tutorial

## Load raw data

In this tutorial, you will clean and validate a dataset containing synthesized product data. The data is loaded from a CSV file into a Pandas DataFrame.

In [2]:
DATA_DIR = pathlib.Path("/cookbooks/data/raw")

df_products_raw = pd.read_csv(DATA_DIR / "products.csv", encoding="unicode_escape")

In [3]:
print(f"Loaded {df_products_raw.shape[0]} product rows into dataframe.\n")

display(df_products_raw.head())

Loaded 2517 product rows into dataframe.



,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio


## Examine destination tables

The product data will be normalized and loaded into multiple Postgres tables:
* `products`
* `product_category`
* `product_subcategory`

Examine the schema of the destination tables and compare to the initial schema and contents of the raw product data.

In [4]:
tutorial.db.get_table_schema(table_name="products")

,column,data_type,nullable,primary_key
0,product_id,bigint,False,True
1,name,text,True,False
2,brand,text,True,False
3,color,text,True,False
4,unit_cost_usd,double precision,True,False
5,unit_price_usd,double precision,True,False
6,product_category_id,bigint,True,False
7,product_subcategory_id,bigint,True,False


In [5]:
tutorial.db.get_table_schema(table_name="product_category")

,column,data_type,nullable,primary_key
0,product_category_id,bigint,False,True
1,name,text,True,False


In [6]:
tutorial.db.get_table_schema(table_name="product_subcategory")

,column,data_type,nullable,primary_key
0,product_subcategory_id,bigint,False,True
1,name,text,True,False


## Clean product data

To clean the product data and separate it into three dataframes to normalize the data, you will use a pre-prepared function, `clean_product_data`. The cleaning code is displayed below, and then invoked to clean the raw product data.

In [7]:
%pycat inspect.getsource(tutorial.cookbook2.clean_product_data)

def clean_product_data(
    df_original: pd.DataFrame,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Clean sample product data for Cookbook 2.

    Returns:
        Tuple of pandas dataframes: product data, product categories, product subcategories
    """

    # Generate a separate copy of original data to transform.
    df_products = df_original.copy()

    # Rename original columns.
    RENAME_COLUMNS = {
        "ProductKey": "product_id",
        "Product Name": "name",
        "Brand": "brand",
        "Color": "color",
        "Unit Cost USD": "unit_cost_usd",
        "Unit Price USD": "unit_price_usd",
        "SubcategoryKey": "product_subcategory_id",
        "Subcategory": "product_subcategory_name",
        "CategoryKey": "product_category_id",
        "Category": "product_category_name",
    }

    df_products = df_products.rename(columns=RENAME_COLUMNS)

    # Clean cost and price figures.
    for currency_col in ["unit_cost_usd", "unit_price_usd"]:
       

In [8]:
df_products, df_product_categories, df_product_subcategories = (
    tutorial.cookbook2.clean_product_data(df_products_raw)
)

In [9]:
print(f"Loaded {df_products.shape[0]} cleaned product rows.\n")

df_products.head()

Loaded 2517 cleaned product rows.



,product_id,name,brand,color,unit_cost_usd,unit_price_usd,product_category_id,product_subcategory_id
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,1,101
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,1,101
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,1,101
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,1,101
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,1,101


In [10]:
print(f"Loaded {df_product_subcategories.shape[0]} cleaned product subcategory rows.\n")

df_product_subcategories.head()

Loaded 32 cleaned product subcategory rows.



,product_subcategory_id,name
0,101,MP4&MP3
1,104,Recording Pen
2,106,Bluetooth Headphones
3,201,Televisions
4,202,VCD & DVD


In [11]:
print(f"Loaded {df_product_categories.shape[0]} cleaned product category rows.\n")

df_product_categories.head()

Loaded 8 cleaned product category rows.



,product_category_id,name
0,1,Audio
1,2,TV and Video
2,3,Computers
3,4,Cameras and camcorders
4,5,Cell phones


## GX data validation workflow

You will validate the cleaned product data using GX prior to loading it into a Postgres database table.

The GX data validation workflow was introduced in [Cookbook 1](Cookbook_1_Validate_data_during_ingestion_happy_path.ipynb), which provided a walkthrough of the following GX components:
* Data Context
* Data Source
* Data Asset
* Batch Definition
* Batch
* Expectation
* Expectation Suite
* Validation Result

This cookbook will extend the GX validation workflow to include the Validation Definition and Checkpoint components, and will further explore the validation metadata returned in the Validation Result.

This tutorial contains concise explanations of GX components and workflows. For more detail, visit the [Introduction to GX Core](https://docs.greatexpectations.io/docs/core/introduction/) in the GX docs.

### Set up the GX validation workflow

This validation will create the following Expectations:
* Expect that the product dataset contains the following columns, in the specified order
* Expect that all product unit prices are at least $1 USD
* Expect that all products have a higher unit price than unit cost

```{admonition} Reminder: Adding GX components to the Data Context
GX components are unique on name. Once a component is created with the Data Context, adding another component with the same name will cause an error. To enable repeated execution of cookbook cells that add GX workflow components, you will see the following pattern:

    try:
        Add a new component(s) to the context
    except:
        Get component(s) from the context by name, or delete and recreate the component(s)
```

In [12]:
# Create the Data Context.
context = gx.get_context()

# Create the Data Source, Data Asset, and Batch Definition.
try:
    data_source = context.data_sources.add_pandas("pandas")
    data_asset = data_source.add_dataframe_asset(name="customer data")
    batch_definition = data_asset.add_batch_definition_whole_dataframe(
        "batch definition"
    )

except:
    data_source = context.data_sources.get("pandas")
    data_asset = data_source.get_asset(name="customer data")
    batch_definition = data_asset.get_batch_definition("batch definition")

# Get the Batch from the Batch Definition.
batch = batch_definition.get_batch(batch_parameters={"dataframe": df_products})

# Create the Expectation Suite.
try:
    expectation_suite = context.suites.add(
        gx.ExpectationSuite(name="product expectations")
    )
except:
    expectation_suite = context.suites.delete(name="product expectations")
    expectation_suite = context.suites.add(
        gx.ExpectationSuite(name="product expectations")
    )

expectations = [
    gxe.ExpectTableColumnsToMatchOrderedList(
        column_list=[
            "product_id",
            "name",
            "brand",
            "color",
            "unit_cost_usd",
            "unit_price_usd",
            "product_category_id",
            "product_subcategory_id",
        ]
    ),
    gxe.ExpectColumnValuesToBeBetween(column="unit_price_usd", min_value=1.0),
    gxe.ExpectColumnPairValuesAToBeGreaterThanB(
        column_A="unit_price_usd", column_B="unit_cost_usd"
    ),
]

for expectation in expectations:
    expectation_suite.add_expectation(expectation)

validation_result = batch.validate(expectation_suite)

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmpd_5m9sv2' for ephemeral docs site
Calculating Metrics: 100%|██████████| 16/16 [00:00<00:00, 781.95it/s]


In [13]:
validation_result["success"]

False

### Extend the validation workflow

A **Validation Definition** pairs a Batch Definition with an Expectation Suite. It defines what data you want to validate using which Expectations.

In [14]:
# Create the Validation Definition.
try:
    validation_definition = context.validation_definitions.add(
        gx.ValidationDefinition(
            name="product validation definition",
            data=batch_definition,
            suite=expectation_suite,
        )
    )
except:
    context.validation_definitions.delete(name="product validation definition")
    validation_definition = context.validation_definitions.add(
        gx.ValidationDefinition(
            name="product validation definition",
            data=batch_definition,
            suite=expectation_suite,
        )
    )

A **Checkpoint** executes data validation based on the specifications of the Validation Definition. Checkpoints also enable actions to be tied to data validation, and 

`unexpected_index_column_names`

Result format: https://docs.greatexpectations.io/docs/core/trigger_actions_based_on_results/choose_a_result_format/

In [15]:
# Create Checkpoint.
try:
    checkpoint = context.checkpoints.add(
        gx.Checkpoint(
            name="checkpoint",
            validation_definitions=[validation_definition],
            result_format={
                "result_format": "COMPLETE",
                # "include_unexpected_rows": True,
                # "exclude_unexpected_values": True,
                "unexpected_index_column_names": ["product_id"],
            },
        )
    )
except:
    context.checkpoints.delete(name="checkpoint")
    checkpoint = context.checkpoints.add(
        gx.Checkpoint(
            name="checkpoint",
            validation_definitions=[validation_definition],
            result_format={
                "result_format": "COMPLETE",
                # "include_unexpected_rows": True,
                # "exclude_unexpected_values": True,
                "unexpected_index_column_names": ["product_id"],
            },
        )
    )

Next, run the Checkpoint. When validating dataframe Data Sources, the dataframe must be supplied to the Checkpoint at runtime.

In [16]:
checkpoint_result = checkpoint.run(batch_parameters={"dataframe": df_products})

Calculating Metrics: 100%|██████████| 16/16 [00:00<00:00, 802.85it/s]


## Examine Validation Result

In [17]:
# Extract the Validation Result object from the Checkpoint results.
validation_result = checkpoint_result.run_results[
    list(checkpoint_result.run_results.keys())[0]
]

In [18]:
validation_result["success"]

False

```
 "statistics": {
    "evaluated_expectations": 3,
    "successful_expectations": 2,
    "unsuccessful_expectations": 1,
    "success_percent": 66.66666666666666
  },
```

In [19]:
expectations_run = validation_result["statistics"]["evaluated_expectations"]
expectations_failed = validation_result["statistics"]["unsuccessful_expectations"]

print(
    f"{expectations_run} Expectations were run, {expectations_failed} Expectations failed."
)

3 Expectations were run, 2 Expectations failed.


In [20]:
failed_expectations = []
for result in validation_result["results"]:
    if result["success"] is True:
        failed_expectations.append(result)

## pull out bad rows

In [21]:
failed_expectation = [
    x
    for x in validation_result["results"]
    if x["expectation_config"]["type"] == "expect_column_values_to_be_between"
][0]
failed_expectation

{
  "success": false,
  "expectation_config": {
    "type": "expect_column_values_to_be_between",
    "kwargs": {
      "batch_id": "pandas-customer data",
      "column": "unit_price_usd",
      "min_value": 1.0
    },
    "meta": {},
    "id": "2ffbd820-c0b8-440c-9903-df36eddbe858"
  },
  "result": {
    "element_count": 2517,
    "unexpected_count": 4,
    "unexpected_percent": 0.15891934843067143,
    "partial_unexpected_list": [
      0.95,
      0.95,
      0.95,
      0.48
    ],
    "unexpected_index_column_names": [
      "product_id"
    ],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.15891934843067143,
    "unexpected_percent_nonmissing": 0.15891934843067143,
    "partial_unexpected_counts": [
      {
        "value": 0.95,
        "count": 3
      },
      {
        "value": 0.48,
        "count": 1
      }
    ],
    "partial_unexpected_index_list": [
      {
        "unit_price_usd": 0.95,
        "product_id": 919
      },
      {

In [22]:
failed_expectation["result"]["unexpected_index_list"]
bad_product_ids = [
    x["product_id"] for x in failed_expectation["result"]["unexpected_index_list"]
]
bad_product_ids

[919, 920, 921, 922]

In [23]:
# Pull out bad rows from original product dataset.
df_products[df_products["product_id"].isin(bad_product_ids)]

,product_id,name,brand,color,unit_cost_usd,unit_price_usd,product_category_id,product_subcategory_id
918,919,SV USB Data Cable E600 Pink,Southridge Video,Pink,0.48,0.95,3,308
919,920,SV USB Data Cable E600 Black,Southridge Video,Black,0.48,0.95,3,308
920,921,SV USB Data Cable E600 Silver,Southridge Video,Silver,0.48,0.95,3,308
921,922,SV USB Data Cable E600 Grey,Southridge Video,Grey,0.95,0.48,3,308


In [24]:
# Drop the bad rows.
df_products_validated = df_products.drop(
    df_products[df_products["product_id"].isin(bad_product_ids)].index
).reset_index(drop=True)

df_products_validated

,product_id,name,brand,color,unit_cost_usd,unit_price_usd,product_category_id,product_subcategory_id
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,1,101
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,1,101
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,1,101
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,1,101
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,1,101
...,...,...,...,...,...,...,...,...
2508,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,43.07,129.99,5,505
2509,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,43.07,129.99,5,505
2510,2515,Contoso In-Line Coupler E180 White,Contoso,White,1.71,3.35,5,505
2511,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,1.71,3.35,5,505


In [25]:
(
    products_validation_result,
    product_category_validation_result,
    product_category_validation_result,
) = tutorial.cookbook2.validate_product_data(
    df_products, df_product_categories, df_product_subcategories
)

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmpj2bzmot1' for ephemeral docs site
Calculating Metrics: 100%|██████████| 2/2 [00:00<00:00, 907.27it/s] 
